In [1]:
import json
task = 'maximum_flow'
filename = task+'_result_llama_dpo_novel.json'
with open(filename, 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [2]:
filename1 = task+'_test.json'
with open(filename1, 'r', encoding='utf-8') as f:
    gt=json.load(f)

In [3]:
print(len(datas))
print(len(gt))
print(gt[0])

300
300
{'input_prompt': 'Given a directed graph as follows:\nNode 2 is connected to nodes 3 (weight: 1), 1 (weight: 8), 8 (weight: 4), 5 (weight: 0).\nNode 3 is connected to node 13 (weight: 6).\nNode 1 is connected to nodes 11 (weight: 1), 12 (weight: 7), 10 (weight: 0), 2 (weight: 0).\nNode 8 is connected to nodes 4 (weight: 9), 11 (weight: 3).\nNode 13 is connected to nodes 10 (weight: 3), 9 (weight: 8), 4 (weight: 1).\nNode 11 is connected to nodes 9 (weight: 4), 10 (weight: 0).\nNode 12 is connected to node 8 (weight: 5).\nNode 5 is connected to nodes 3 (weight: 1), 6 (weight: 8), 2 (weight: 4), 0 (weight: 0).\nNode 6 is connected to nodes 0 (weight: 5), 13 (weight: 2), 3 (weight: 3), 9 (weight: 0).\nNode 0 is connected to nodes 7 (weight: 7), 5 (weight: 5), 6 (weight: 0).\nNode 10 is connected to nodes 1 (weight: 2), 11 (weight: 8), 13 (weight: 3).\nNode 9 is connected to nodes 6 (weight: 10), 4 (weight: 3), 12 (weight: 7), 11 (weight: 0).\nNode 7 is connected to nodes 4 (weight

In [4]:
import re
import io
import sys


In [5]:
print(datas[2]['code'])

```python
from collections import defaultdict

class Graph:
    def __init__(self, graph):
        self.graph = graph
        self.ROW = len(graph)

    def bfs(self, s, t, parent):
        visited = [False] * self.ROW
        queue = []
        queue.append(s)
        visited[s] = True

        while queue:
            u = queue.pop(0)
            for ind, val in enumerate(self.graph[u]):
                if visited[ind] == False and val > 0:
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u

        return True if visited[t] else False

    def ford_fulkerson(self, source, sink):
        parent = [-1] * self.ROW
        max_flow = 0

        while self.bfs(source, sink, parent):
            path_flow = float("Inf")
            s = sink
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            max_flow += path_flow

            v = sink
  

In [6]:
error_cnt = 0
correct_cnt = 0
for i, data in enumerate(datas):
    text = data['code']
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    ans = gt[i]['answer']

    # 创建一个 StringIO 对象来捕获输出
    output = io.StringIO()

    # 重定向 sys.stdout
    old_stdout = sys.stdout
    sys.stdout = output

    try:
        exec(code, globals(), globals())
    except:
        sys.stdout = old_stdout
        print(code)
        error_cnt = error_cnt + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    # 获取输出内容
    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    #print(output_content)
    
    if output_content.strip() == str(ans).strip():
        correct_cnt = correct_cnt + 1
    else:
        print(output_content,ans)
    
    
    # print(f'llm={output_content} gt={ans}')
    #output_content = float(output_content)
    # if round(output_content, 3) == round(ans, 3):
    #     correct_cnt = correct_cnt + 1
    # else:
    #     #print(code)
    #     print(output_content,ans)
    #     print(code)

The maximum possible flow is 0 6
The maximum possible flow is 0 17
The maximum possible flow is 2 7
The maximum possible flow is 3 3
The maximum possible flow is 3 16
The maximum possible flow is 0 9
The maximum possible flow is 0 4
The maximum possible flow is 11 11
The maximum possible flow is 0 20
The maximum possible flow is 0 9
The maximum possible flow is 3 9
The maximum possible flow is 0 16
The maximum possible flow is 9 21
The maximum possible flow is 0 21
The maximum possible flow is 0 8
The maximum possible flow is 0 3
The maximum possible flow is 0 10
The maximum possible flow is 0 9
The maximum possible flow is 5 5
The maximum possible flow is 4 1
The maximum possible flow is 1 5
The maximum possible flow is 0 8
The maximum possible flow is 10 13
The maximum possible flow is 0 18
The maximum possible flow is 0 2
The maximum possible flow is 0 22
The maximum possible flow is 1 7
The maximum possible flow is 7 7
The maximum possible flow is 0 11
The maximum possible flow is 

In [7]:
print(error_cnt)
print(correct_cnt)

21
0


In [8]:
print((correct_cnt+20)/len(datas))
#max tokens是

0.06666666666666667


In [10]:
import re
import networkx as nx

# Graph description
graph = """Node 7 is connected to nodes 12, 3, 2, 0.
Node 12 is connected to nodes 5, 6, 10, 11, 4.
Node 3 is connected to nodes 2, 11, 0, 4, 1, 8.
Node 2 is connected to nodes 10, 0, 7, 3.
Node 0 is connected to nodes 7, 13, 6, 8, 3, 2, 11.
Node 9 is connected to nodes 4, 13, 12, 6, 3, 10, 11.
Node 4 is connected to nodes 13, 5, 1, 2, 10, 11, 7, 9.
Node 13 is connected to nodes 5, 6, 8, 3, 0.
Node 6 is connected to nodes 2, 11, 0, 4, 12.
Node 10 is connected to nodes 0, 9, 6.
Node 11 is connected to nodes 0, 7, 13, 12, 5, 6, 2, 10.
Node 5 is connected to node 6.
Node 1 is connected to nodes 8, 2, 11, 0, 4.
Node 8 is connected to nodes 11, 0, 7, 13, 6."""

# Regular expression to extract connections
pattern = r'Node (\d+) is connected to nodes? ([\d, ]+)\.'
matches = re.findall(pattern, graph)

# Create a directed graph
G = nx.Graph()

# Add edges to the graph
for node, neighbors in matches:
    node = int(node)
    neighbors = map(int, neighbors.split(', '))
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

# Calculate Jaccard coefficient
node1 = 0
node2 = 10
preds = nx.jaccard_coefficient(G, [(node1, node2)])
print(preds)


<generator object _apply_prediction.<locals>.<genexpr> at 0x7f57040f4640>
